# Agenda

1. Recap
2. Address book
3. More with reading from and writing to files
4. Cleaning data with `nan` and interpolating
5. Analysis with data frames
    - Cutting and categorizing
    - Sorting
    - Grouping
    - Concatenating data frames together
    - Join data frames
    

# Recap

When we use Pandas, we're mainly using two different data structures:

- Series, which is basically a 1D NumPy array with a nice set of wrappers around it.  Each series has a single dtype.  Pandas often guesses correctly, but you can set it just as you did with NumPy arrays.
- Data frame, which is basically a glorified 2D NumPy array.  Each column in a data frame is a separate series, which means that each column has a separate dtype.  

Both a series and a data frame have an *index*, which describes the rows. An index can contain any type of values at all -- integers, strings, dates, or anything else.  Integers and strings are most common.  The values can even repeat.

A data frame, in addition to an index, has a value for "columns," which describes the names of the columns.

We can retrieve from either a series or from a data frame via the index using `.loc`.  Or we can use the numeric position using `.iloc`.

In [1]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

In [3]:
df = DataFrame(np.random.randint(0, 1000, [5,6]),
              index=list('vwxyz'),      # rows
              columns=list('abcdef'))   # columns
df

,a,b,c,d,e,f
v,772,582,393,320,11,773
w,400,535,723,139,423,244
x,475,892,999,438,333,382
y,610,323,559,372,365,336
z,770,201,77,18,935,138


In [4]:
# I can retrieve an entire row via .loc and an index

df.loc['x']

a    475
b    892
c    999
d    438
e    333
f    382
Name: x, dtype: int64

In [5]:
df.loc['x', 'd']   # retrieve row x, column d

438

In [6]:
df.loc['x', 'd'] = 12.34
df   # the dtype for d has changed - now it's np.float64

,a,b,c,d,e,f
v,772,582,393,320.00,11,773
w,400,535,723,139.00,423,244
x,475,892,999,12.34,333,382
y,610,323,559,372.00,365,336
z,770,201,77,18.00,935,138


In [7]:
df.dtypes  # show me all dtypes for all columns

a      int64
b      int64
c      int64
d    float64
e      int64
f      int64
dtype: object

In [8]:
# d is now a float64 column
# but what if I retrieve row x?

df.loc['x']   # the dtype of this row is float64, because Pandas needs to find a type that's good for all values

a    475.00
b    892.00
c    999.00
d     12.34
e    333.00
f    382.00
Name: x, dtype: float64

In [10]:
# what if I want to find all of the elements of column b that are even?

df['b']%2

v    0
w    1
x    0
y    1
z    1
Name: b, dtype: int64

In [11]:
df['b']%2 == 0   # the remainder is 0 if the numbers are even

v     True
w    False
x     True
y    False
z    False
Name: b, dtype: bool

In [12]:
# I can apply this boolean series as a mask index on df['b']
# in this way, I can get a new series, containing all of the values of df['b']
# that are even

#        apply this boolean series as a mask
df['b'][df['b']%2 == 0]

v    582
x    892
Name: b, dtype: int64

In [13]:
# what if we apply our mask index not only to df['b'], but to all of df?

# this will show me all of the rows of the data frame
# (all columns) where b is even 
# aka: it'll only show us rows v and x of df
df[df['b']%2 == 0]

,a,b,c,d,e,f
v,772,582,393,320.00,11,773
x,475,892,999,12.34,333,382


In [14]:
# if we use .loc, and don't directly apply [] to df, we 
# can then also specify which columns we want

# that's because df.loc has the syntax of
# df.loc[ROW_SELECTOR, COLUMN_SELECTOR]
# if you don't select columns explicitly, then you get all of them.

df.loc[df['b']%2 == 0]

,a,b,c,d,e,f
v,772,582,393,320.00,11,773
x,475,892,999,12.34,333,382


In [15]:
# this shows all rows of df
# where df['b'] is even
# and only column 'c'

df.loc[df['b']%2 == 0, 'c']

v    393
x    999
Name: c, dtype: int64

In [16]:
# all rows of df
# where df['b'] is even
# and only columns c and e

df.loc[df['b']%2 == 0, ['c', 'e']]

,c,e
v,393,11
x,999,333


In [19]:
# show me all rows of df
# where df['c'] < df['c'].mean()
# and only columns a and d

df.loc[df['c'] < df['c'].mean(), ['a', 'd']]

,a,d
v,772,320.0
z,770,18.0


In [20]:
df.describe()

,a,b,c,d,e,f
count,5.000000,5.000000,5.00000,5.00000,5.000000,5.00000
mean,605.400000,506.600000,550.20000,172.26800,413.400000,374.60000
std,168.866219,265.577296,346.40612,167.48131,332.750357,241.50735
min,400.000000,201.000000,77.00000,12.34000,11.000000,138.00000
25%,475.000000,323.000000,393.00000,18.00000,333.000000,244.00000
50%,610.000000,535.000000,559.00000,139.00000,365.000000,336.00000
75%,770.000000,582.000000,723.00000,320.00000,423.000000,382.00000
max,772.000000,892.000000,999.00000,372.00000,935.000000,773.00000


In [21]:
df.mean()

a    605.400
b    506.600
c    550.200
d    172.268
e    413.400
f    374.600
dtype: float64

In [22]:
df.sum()

a    3027.00
b    2533.00
c    2751.00
d     861.34
e    2067.00
f    1873.00
dtype: float64

In [23]:
df.max()

a    772.0
b    892.0
c    999.0
d    372.0
e    935.0
f    773.0
dtype: float64

In [24]:
df

,a,b,c,d,e,f
v,772,582,393,320.00,11,773
w,400,535,723,139.00,423,244
x,475,892,999,12.34,333,382
y,610,323,559,372.00,365,336
z,770,201,77,18.00,935,138


In [25]:
df['g'] = ['duck', 'duck', 'duck', 'duck', 'goose']

In [26]:
df

,a,b,c,d,e,f,g
v,772,582,393,320.00,11,773,duck
w,400,535,723,139.00,423,244,duck
x,475,892,999,12.34,333,382,duck
y,610,323,559,372.00,365,336,duck
z,770,201,77,18.00,935,138,goose


In [27]:
df.describe()

,a,b,c,d,e,f
count,5.000000,5.000000,5.00000,5.00000,5.000000,5.00000
mean,605.400000,506.600000,550.20000,172.26800,413.400000,374.60000
std,168.866219,265.577296,346.40612,167.48131,332.750357,241.50735
min,400.000000,201.000000,77.00000,12.34000,11.000000,138.00000
25%,475.000000,323.000000,393.00000,18.00000,333.000000,244.00000
50%,610.000000,535.000000,559.00000,139.00000,365.000000,336.00000
75%,770.000000,582.000000,723.00000,320.00000,423.000000,382.00000
max,772.000000,892.000000,999.00000,372.00000,935.000000,773.00000


In [28]:
df['g'].describe()

count        5
unique       2
top       duck
freq         4
Name: g, dtype: object

# Exercise: Address book

1. Create a data frame in which you have a few friends and family members. Every person in the data frame will have the following columns:
    - `firstname`
    - `lastname`
    - `age`
2. Create the data frame with about 7-10 people.
3. What is the average age of people in your address book?
4. Show the first and last names of people whose ages are above average.
5. Show people (name and age) whose first name is shorter than the average for first names.

In [29]:
df = DataFrame([['a', 'b', 10],
                ['c', 'd', 20],
                ['e', 'f', 30]])
df

,0,1,2
0,a,b,10
1,c,d,20
2,e,f,30


In [32]:
df = DataFrame([['Reuven', 'Lerner', 51],
                ['Atara', 'Lerner-Friedman', 21],
                ['Shikma', 'Lerner-Friedman', 19],
                ['Amotz', 'Lerner-Friedman', 16],
                ['John', 'Smith', 35],
                ['David', 'Cohen', 60],
                ['Sarah', 'Friedman', 59]                
               ],
              columns='firstname lastname age'.split())     # ['firstname', 'lastname', 'age']

In [33]:
df

,firstname,lastname,age
0,Reuven,Lerner,51
1,Atara,Lerner-Friedman,21
2,Shikma,Lerner-Friedman,19
3,Amotz,Lerner-Friedman,16
4,John,Smith,35
5,David,Cohen,60
6,Sarah,Friedman,59


In [35]:
# what's the average age of people in my address book?

df['age'].mean()

37.285714285714285

In [36]:
# what are the names of people whose ages are above average

df['age'] > df['age'].mean()   # this returns a boolean series

0     True
1    False
2    False
3    False
4    False
5     True
6     True
Name: age, dtype: bool

In [37]:
#        row selectors via a mask/boolean index
df.loc[df['age'] > df['age'].mean()]

,firstname,lastname,age
0,Reuven,Lerner,51
5,David,Cohen,60
6,Sarah,Friedman,59


In [38]:
#       row selector                    ,  column selector
df.loc[df['age'] > df['age'].mean(),      ['firstname', 'lastname']  ]

,firstname,lastname
0,Reuven,Lerner
5,David,Cohen
6,Sarah,Friedman


In [41]:
# find all people
# whose first name is shorter than the average first name

df['firstname'].str.len().mean()

5.142857142857143

In [42]:
df['firstname'].str.len() < df['firstname'].str.len().mean()

0    False
1     True
2    False
3     True
4     True
5     True
6     True
Name: firstname, dtype: bool

In [44]:
# find all rows
# where the first name is shorter than the average first name
# all columns (so we don't need a column selector)

df.loc[df['firstname'].str.len() < df['firstname'].str.len().mean()]

,firstname,lastname,age
1,Atara,Lerner-Friedman,21
3,Amotz,Lerner-Friedman,16
4,John,Smith,35
5,David,Cohen,60
6,Sarah,Friedman,59


In [45]:
# earlier, I created the data frame as a list of lists

df = DataFrame([['Reuven', 'Lerner', 51],
                ['Atara', 'Lerner-Friedman', 21],
                ['Shikma', 'Lerner-Friedman', 19],
                ['Amotz', 'Lerner-Friedman', 16],
                ['John', 'Smith', 35],
                ['David', 'Cohen', 60],
                ['Sarah', 'Friedman', 59]                
               ],
              columns='firstname lastname age'.split())     # ['firstname', 'lastname', 'age']

In [46]:
# I can also create this data frame as a list of dicts
# each dictionary represents one row
# the keys are the column names, and the values are .. the values

# we don't need to specify column names


df = DataFrame([{'firstname':'Reuven', 'lastname':'Lerner', 'age':51},
               {'firstname':'Atara', 'lastname':'Lerner-Friedman', 'age':21},
               {'firstname':'Shikma', 'lastname':'Lerner-Friedman', 'age':19},
               {'firstname':'Amotz', 'lastname':'Lerner-Friedman', 'age':16}
               ])

In [47]:
df

,firstname,lastname,age
0,Reuven,Lerner,51
1,Atara,Lerner-Friedman,21
2,Shikma,Lerner-Friedman,19
3,Amotz,Lerner-Friedman,16


# Reading from and writing to files

Yesterday, we saw that we can read from CSV, Excel, and feather files.  We can write to them, as well.

We saw a few of the parameters we can specify when reading from a CSV file:
- `sep`, the separator, defaulting to `,`
- `usecols`, a list of column names that we want to include in our data frame

In [49]:
!head airlines.dat

1,"Private flight",\N,"-","N/A","","","Y" 
2,"135 Airways",\N,"","GNL","GENERAL","United States","N"
3,"1Time Airline",\N,"1T","RNX","NEXTIME","South Africa","Y"
4,"2 Sqn No 1 Elementary Flying Training School",\N,"","WYT","","United Kingdom","N"
5,"213 Flight Unit",\N,"","TFU","","Russia","N"
6,"223 Flight Unit State Airline",\N,"","CHD","CHKALOVSK-AVIA","Russia","N"
7,"224th Flight Unit",\N,"","TTF","CARGO UNIT","Russia","N"
8,"247 Jet Ltd",\N,"","TWF","CLOUD RUNNER","United Kingdom","N"
9,"3D Aviation",\N,"","SEC","SECUREX","United States","N"
10,"40-Mile Air",\N,"Q5","MLA","MILE-AIR","United States","Y"


In [50]:
# read in data about every airline in the world
df = pd.read_csv('airlines.dat')

In [51]:
df.head()

,1,Private flight,\N,-,N/A,Unnamed: 5,Unnamed: 6,Y
0,2,135 Airways,\N,NaN,GNL,GENERAL,United States,N
1,3,1Time Airline,\N,1T,RNX,NEXTIME,South Africa,Y
2,4,2 Sqn No 1 Elementary Flying Training School,\N,NaN,WYT,NaN,United Kingdom,N
3,5,213 Flight Unit,\N,NaN,TFU,NaN,Russia,N
4,6,223 Flight Unit State Airline,\N,NaN,CHD,CHKALOVSK-AVIA,Russia,N


In [69]:
# if the CSV file doesn't start with column names, we need to:
# (1) tell it not to use the first row as columns, so we don't lose data
# (2) name the columns ourselves

df = pd.read_csv('airlines.dat', 
                header=None,   # the first row of the file is *not* a header row
                names=['id', 'fullname', 'junk', '2code', '3code', 'formal name', 'country', 'morejunk'],
                usecols=['id', 'fullname', '2code', '3code', 'formal name', 'country'])

In [70]:
df.head()

,id,fullname,2code,3code,formal name,country
0,1,Private flight,-,NaN,NaN,NaN
1,2,135 Airways,NaN,GNL,GENERAL,United States
2,3,1Time Airline,1T,RNX,NEXTIME,South Africa
3,4,2 Sqn No 1 Elementary Flying Training School,NaN,WYT,NaN,United Kingdom
4,5,213 Flight Unit,NaN,TFU,NaN,Russia


In [66]:
'abc', 'def'

('abc', 'def')

In [67]:
'abc' 'def'

'abcdef'

In [71]:
# I'd like the "id" column to be the index of my data frame

df = df.set_index('id')   # set_index returns a new data frame, in which the "id" column is the index

In [72]:
df

,fullname,2code,3code,formal name,country
id,,,,,
1,Private flight,-,NaN,NaN,NaN
2,135 Airways,NaN,GNL,GENERAL,United States
3,1Time Airline,1T,RNX,NEXTIME,South Africa
4,2 Sqn No 1 Elementary Flying Training School,NaN,WYT,NaN,United Kingdom
5,213 Flight Unit,NaN,TFU,NaN,Russia
...,...,...,...,...,...
19828,Vuela Cuba,6C,6CC,NaN,Cuba
19830,All Australia,88,8K8,NaN,Australia
19831,Fly Europa,ER,RWW,NaN,Spain


In [73]:
df = df.reset_index()   # return a new data frame without an index, and then assign it to df

In [74]:
df

,id,fullname,2code,3code,formal name,country
0,1,Private flight,-,NaN,NaN,NaN
1,2,135 Airways,NaN,GNL,GENERAL,United States
2,3,1Time Airline,1T,RNX,NEXTIME,South Africa
3,4,2 Sqn No 1 Elementary Flying Training School,NaN,WYT,NaN,United Kingdom
4,5,213 Flight Unit,NaN,TFU,NaN,Russia
...,...,...,...,...,...,...
6043,19828,Vuela Cuba,6C,6CC,NaN,Cuba
6044,19830,All Australia,88,8K8,NaN,Australia
6045,19831,Fly Europa,ER,RWW,NaN,Spain
6046,19834,FlyPortugal,PO,FPT,FlyPortugal,Portugal


In [75]:
df = pd.read_csv('airlines.dat', 
                header=None,   # the first row of the file is *not* a header row
                names=['id', 'fullname', 'junk', '2code', '3code', 'formal name', 'country', 'morejunk'],
                usecols=['id', 'fullname', '2code', '3code', 'formal name', 'country'],
                index_col='id')
df

,fullname,2code,3code,formal name,country
id,,,,,
1,Private flight,-,NaN,NaN,NaN
2,135 Airways,NaN,GNL,GENERAL,United States
3,1Time Airline,1T,RNX,NEXTIME,South Africa
4,2 Sqn No 1 Elementary Flying Training School,NaN,WYT,NaN,United Kingdom
5,213 Flight Unit,NaN,TFU,NaN,Russia
...,...,...,...,...,...
19828,Vuela Cuba,6C,6CC,NaN,Cuba
19830,All Australia,88,8K8,NaN,Australia
19831,Fly Europa,ER,RWW,NaN,Spain


In [76]:
df = DataFrame([['Reuven', 'Lerner', 51],
                ['Atara', 'Lerner-Friedman', 21],
                ['Shikma', 'Lerner-Friedman', 19],
                ['Amotz', 'Lerner-Friedman', 16],
                ['John', 'Smith', 35],
                ['David', 'Cohen', 60],
                ['Sarah', 'Friedman', 59]                
               ],
              columns='פרטי משפחה גיל'.split())     # ['firstname', 'lastname', 'age']

In [77]:
df.head()

,פרטי,משפחה,גיל
0,Reuven,Lerner,51
1,Atara,Lerner-Friedman,21
2,Shikma,Lerner-Friedman,19
3,Amotz,Lerner-Friedman,16
4,John,Smith,35


In [78]:
df.to_csv('family.csv')

In [79]:
!cat family.csv

,פרטי,משפחה,גיל
0,Reuven,Lerner,51
1,Atara,Lerner-Friedman,21
2,Shikma,Lerner-Friedman,19
3,Amotz,Lerner-Friedman,16
4,John,Smith,35
5,David,Cohen,60
6,Sarah,Friedman,59


In [80]:
df = pd.read_csv('family.csv')

In [81]:
df

,Unnamed: 0,פרטי,משפחה,גיל
0,0,Reuven,Lerner,51
1,1,Atara,Lerner-Friedman,21
2,2,Shikma,Lerner-Friedman,19
3,3,Amotz,Lerner-Friedman,16
4,4,John,Smith,35
5,5,David,Cohen,60
6,6,Sarah,Friedman,59


In [82]:
df.drop('Unnamed: 0', axis='columns')

,פרטי,משפחה,גיל
0,Reuven,Lerner,51
1,Atara,Lerner-Friedman,21
2,Shikma,Lerner-Friedman,19
3,Amotz,Lerner-Friedman,16
4,John,Smith,35
5,David,Cohen,60
6,Sarah,Friedman,59


In [83]:
df.גיל

0    51
1    21
2    19
3    16
4    35
5    60
6    59
Name: גיל, dtype: int64

In [84]:
df.to_json('family.json')

In [85]:
!cat family.json

{"Unnamed: 0":{"0":0,"1":1,"2":2,"3":3,"4":4,"5":5,"6":6},"\u05e4\u05e8\u05d8\u05d9":{"0":"Reuven","1":"Atara","2":"Shikma","3":"Amotz","4":"John","5":"David","6":"Sarah"},"\u05de\u05e9\u05e4\u05d7\u05d4":{"0":"Lerner","1":"Lerner-Friedman","2":"Lerner-Friedman","3":"Lerner-Friedman","4":"Smith","5":"Cohen","6":"Friedman"},"\u05d2\u05d9\u05dc":{"0":51,"1":21,"2":19,"3":16,"4":35,"5":60,"6":59}}

In [86]:
df = pd.read_json('family.json')

In [87]:
df

,Unnamed: 0,פרטי,משפחה,גיל
0,0,Reuven,Lerner,51
1,1,Atara,Lerner-Friedman,21
2,2,Shikma,Lerner-Friedman,19
3,3,Amotz,Lerner-Friedman,16
4,4,John,Smith,35
5,5,David,Cohen,60
6,6,Sarah,Friedman,59


In [88]:
help(pd.read_json)

Help on function read_json in module pandas.io.json._json:

read_json(path_or_buf=None, orient=None, typ='frame', dtype: 'DtypeArg | None' = None, convert_axes=None, convert_dates=True, keep_default_dates: 'bool' = True, numpy: 'bool' = False, precise_float: 'bool' = False, date_unit=None, encoding=None, encoding_errors: 'str | None' = 'strict', lines: 'bool' = False, chunksize: 'int | None' = None, compression: 'CompressionOptions' = 'infer', nrows: 'int | None' = None, storage_options: 'StorageOptions' = None)
    Convert a JSON string to pandas object.
    
    Parameters
    ----------
    path_or_buf : a valid JSON str, path object or file-like object
        Any valid string path is acceptable. The string could be a URL. Valid
        URL schemes include http, ftp, s3, and file. For file URLs, a host is
        expected. A local file could be:
        ``file://localhost/path/to/table.json``.
    
        If you want to pass in a path object, pandas accepts any
        ``os.PathLi

# Exercise: Big cities

1. Read `cities.json` into a data frame. (I'll give you the URL for this file, containing the 1,000 largest cities in the United States.)

Don't forget that `pd.read_csv` and `pd.read_json` take URLs, not just filenames, as a first argument.

2. What are the mean and median populations for these cities?
4. What if we remove the 50 most populous cities from these calculations? Does the mean change?  Does the median change?  (You can get this by looking for a rank of >=50.)
5. What is the northernmost city in this data set?
6. Which state has the most cities in this list? Which has the fewest?

In [89]:
url = 'https://gist.githubusercontent.com/reuven/77edbb0292901f35019f17edb9794358/raw/2bf258763cdddd704f8ffd3ea9a3e81d25e2c6f6/cities.json'

df = pd.read_json(url)

In [90]:
df.head()

,city,growth_from_2000_to_2013,latitude,longitude,population,rank,state
0,New York,4.8%,40.712784,-74.005941,8405837,1,New York
1,Los Angeles,4.8%,34.052234,-118.243685,3884307,2,California
2,Chicago,-6.1%,41.878114,-87.629798,2718782,3,Illinois
3,Houston,11.0%,29.760427,-95.369803,2195914,4,Texas
4,Philadelphia,2.6%,39.952584,-75.165222,1553165,5,Pennsylvania


In [91]:
# what are the mean and median populations for these cities?

df['population'].mean()

131132.443

In [92]:
df['population'].median()

68207.0

In [93]:
df['population'].quantile(0.5)

68207.0

In [94]:
df['population'].describe()

count    1.000000e+03
mean     1.311324e+05
std      3.416902e+05
min      3.687700e+04
25%      4.969775e+04
50%      6.820700e+04
75%      1.098850e+05
max      8.405837e+06
Name: population, dtype: float64

In [98]:
df['population'].agg(['mean', 'median'])

mean      131132.443
median     68207.000
Name: population, dtype: float64

In [102]:
# remove the 50 most populous cities.  Do the mean and median change?
#       row selector ,     column selector 
df.loc[df['rank'] > 50,   'population'].agg(['mean', 'median'])

mean      87027.387368
median    65796.000000
Name: population, dtype: float64

In [103]:
df.loc[df['rank'] > 100,   'population'].agg(['mean', 'median'])

mean      76823.707778
median    63308.500000
Name: population, dtype: float64

In [104]:
df.loc[df['rank'] > 100].head()

,city,growth_from_2000_to_2013,latitude,longitude,population,rank,state
100,Birmingham,-12.3%,33.520661,-86.802490,212113,101,Alabama
101,Spokane,7.0%,47.658780,-117.426047,210721,102,Washington
102,Rochester,-4.1%,43.161030,-77.610922,210358,103,New York
103,Des Moines,3.9%,41.600545,-93.609106,207510,104,Iowa
104,Modesto,7.7%,37.639097,-120.996878,204933,105,California


In [106]:
# what is the northernmost city in this data set?
# this would mean the greatest number for latitude

df.loc[df['latitude'] == df['latitude'].max()]

,city,growth_from_2000_to_2013,latitude,longitude,population,rank,state
62,Anchorage,15.4%,61.218056,-149.900278,300950,63,Alaska


In [107]:
# which state has the most (and the fewest) cities in our "big city" list?

df['state'].value_counts()

California              212
Texas                    83
Florida                  73
Illinois                 52
Massachusetts            36
Ohio                     33
Michigan                 31
Washington               28
Arizona                  25
Minnesota                24
North Carolina           22
New Jersey               22
Indiana                  21
Colorado                 21
Wisconsin                20
Utah                     19
Georgia                  18
Virginia                 17
New York                 17
Tennessee                17
Missouri                 16
Connecticut              15
Oregon                   14
Pennsylvania             13
Iowa                     13
South Carolina           12
Alabama                  12
Kansas                   11
Oklahoma                 11
Arkansas                 10
Louisiana                 9
Idaho                     8
Maryland                  7
New Mexico                7
Mississippi               6
Rhode Island        

# Next up

1. Cleaning data: Removing `nan` and interpolating
2. Analysis tools
    - Cutting
    - Categories
    - Sorting
    - Grouping

Resume at :50

In [108]:
np.random.seed(0)
df = DataFrame(np.random.randint(0, 100, [5,4]),
              index=list('vwxyz'),
              columns=list('abcd'))
df

,a,b,c,d
v,44,47,64,67
w,67,9,83,21
x,36,87,70,88
y,88,12,58,65
z,39,87,46,88


In [109]:
df.loc['v', ['c', 'd']] = np.nan
df.loc['x', ['a', 'b', 'c']] = np.nan
df.loc['z', 'd'] = np.nan


In [110]:
df

,a,b,c,d
v,44.0,47.0,NaN,NaN
w,67.0,9.0,83.0,21.0
x,NaN,NaN,NaN,88.0
y,88.0,12.0,58.0,65.0
z,39.0,87.0,46.0,NaN


In [111]:
df.describe()

,a,b,c,d
count,4.000000,4.00,3.000000,3.000000
mean,59.500000,38.75,62.333333,58.000000
std,22.575798,36.50,18.876794,34.044089
min,39.000000,9.00,46.000000,21.000000
25%,42.750000,11.25,52.000000,43.000000
50%,55.500000,29.50,58.000000,65.000000
75%,72.250000,57.00,70.500000,76.500000
max,88.000000,87.00,83.000000,88.000000


In [112]:
# how can I clean these values up?
# option 1: replace with another value
# option 1a: replace with the mean

# I don't want to replace columns a, b, c, and d's nans with the *same* value.
# I want to replace nans in a with a's mean, and nans in b with b's mean, etc.

df.fillna(5)  # all nan values in df are now replaced with 5

,a,b,c,d
v,44.0,47.0,5.0,5.0
w,67.0,9.0,83.0,21.0
x,5.0,5.0,5.0,88.0
y,88.0,12.0,58.0,65.0
z,39.0,87.0,46.0,5.0


In [113]:
df.mean()  # running a series method on the data frame returns a series, the mean for each column

a    59.500000
b    38.750000
c    62.333333
d    58.000000
dtype: float64

In [114]:
df.fillna(df.mean())  # passing a series to fillna then uses the series to fill the nan values

,a,b,c,d
v,44.0,47.00,62.333333,58.0
w,67.0,9.00,83.000000,21.0
x,59.5,38.75,62.333333,88.0
y,88.0,12.00,58.000000,65.0
z,39.0,87.00,46.000000,58.0


In [115]:
# option 1b: the mean can be skewed too much by outliers
# let's instead use the median

df.fillna(df.median())

,a,b,c,d
v,44.0,47.0,58.0,65.0
w,67.0,9.0,83.0,21.0
x,55.5,29.5,58.0,88.0
y,88.0,12.0,58.0,65.0
z,39.0,87.0,46.0,65.0


# Interpolation

Interpolation means: If we have a missing value, we'll assume that it probably lies between the value before it and the value after it.

For example, if a stock price is $100 on Monday, $110 on Tuesday, and $130 on Thursday, we can probably assume that it was $120 on Wednesday.  That's what interpolation tries to do -- it assumes that we can reasonably fill in nan values with the average (mean) of the earlier and later values.

In [117]:
df

,a,b,c,d
v,44.0,47.0,NaN,NaN
w,67.0,9.0,83.0,21.0
x,NaN,NaN,NaN,88.0
y,88.0,12.0,58.0,65.0
z,39.0,87.0,46.0,NaN


In [118]:
df.interpolate()

,a,b,c,d
v,44.0,47.0,NaN,NaN
w,67.0,9.0,83.0,21.0
x,77.5,10.5,70.5,88.0
y,88.0,12.0,58.0,65.0
z,39.0,87.0,46.0,65.0


In [119]:
(67 + 88)/ 2

77.5

In [125]:
df.dropna()   # get rid of all rows with any NaN values

,a,b,c,d
w,67.0,9.0,83.0,21.0
y,88.0,12.0,58.0,65.0


In [126]:
df.dropna(thresh=3)  # if we have at least 3 values, then keep the row

,a,b,c,d
w,67.0,9.0,83.0,21.0
y,88.0,12.0,58.0,65.0
z,39.0,87.0,46.0,NaN


In [127]:
# in the zipfile is a CSV file with all New York City parking tickets from 2020



In [128]:
!head ../data/nyc-parking-violations-2020.csv

Summons Number,Plate ID,Registration State,Plate Type,Issue Date,Violation Code,Vehicle Body Type,Vehicle Make,Issuing Agency,Street Code1,Street Code2,Street Code3,Vehicle Expiration Date,Violation Location,Violation Precinct,Issuer Precinct,Issuer Code,Issuer Command,Issuer Squad,Violation Time,Time First Observed,Violation County,Violation In Front Of Or Opposite,House Number,Street Name,Intersecting Street,Date First Observed,Law Section,Sub Division,Violation Legal Code,Days Parking In Effect,From Hours In Effect,To Hours In Effect,Vehicle Color,Unregistered Vehicle?,Vehicle Year,Meter Number,Feet From Curb,Violation Post Code,Violation Description,No Standing or Stopping Violation,Hydrant Violation,Double Parking Violation
1477633194,J58JKX,NJ,PAS,05/08/1972 12:00:00 AM,16,SDN,HONDA,P,8730,5130,5280,0,72,72,504,342924,T504,0000,0523P,,K,F,270,43 ST,,0,408,E2,,YYYYYBB,0800A,0400P,BK,0,0,-,0,,,,,
1449715424,KRE6058,PA,PAS,08/29/1977 12:00:00 AM,98,SUBN,ME/BE,P,86530,71800,73110,0,7

# Exercise: Parking cleanup

1. Create a data frame from the NYC parking data file.  We only need a handful of columns: Plate ID, registration state, vehicle make, vehicle color, violation time, and street name.
2. How many rows are in the data frame?  
3. Which vehicle color has the most violations?
4. Which vehicle make has the most violations?
5. What are the 10 least common vehicle colors cited here?  Are these mistaken colors? How would such mistakes affect our data?
6. If you remove all of the rows in which we have nans, how many rows are you removing? If each violation is $100 toward NYC's budget, how much money would they lose based on missing data?
7. Remove rows that are missing at least 3 columns worth of data.  How many rows have you removed now?
8. Which of the columns in the data frame contains the greatest number of nan values?

In [130]:
filename = '../data/nyc-parking-violations-2020.csv'

!ls -lh ../data/nyc-parking-violations-2020.csv

-rw-r--r-- 1 reuven staff 2.2G Jul  5  2021 ../data/nyc-parking-violations-2020.csv


In [131]:
filename = '../data/nyc-parking-violations-2020.csv'

df = pd.read_csv(filename,
                usecols=['Plate ID', 'Registration State', 'Vehicle Make', 'Vehicle Color', 
                        'Violation Time', 'Street Name'])

In [132]:
df.shape

(12495734, 6)

In [134]:
df.memory_usage(deep=True).sum()

4693114747

In [136]:
# Which vehicle color has the most violations?

df['Vehicle Color'].value_counts().head(20)

WH       2344858
GY       2307704
BK       2066374
WHITE    1061234
BL        775124
RD        483298
BLACK     465110
GREY      306787
BROWN     292348
SILVE     191477
GR        182929
BLUE      178298
RED       161693
TN        120576
BR        102204
YW         98700
BLK        91539
OTHER      60245
GREEN      58765
GL         54851
Name: Vehicle Color, dtype: int64

In [137]:
df['Vehicle Make'].value_counts().head(20)

TOYOT    1395273
HONDA    1343265
FORD     1328063
NISSA    1119587
CHEVR     711464
FRUEH     530846
ME/BE     530473
JEEP      490977
BMW       488545
DODGE     462646
HYUND     357747
LEXUS     293752
ACURA     247954
INTER     231149
INFIN     230237
GMC       214300
SUBAR     206609
VOLKS     192747
AUDI      182256
HIN       181538
Name: Vehicle Make, dtype: int64

In [138]:
df['Vehicle Color'].value_counts().tail(20)

CHE      1
ORGL     1
BUI      1
GROON    1
HWT      1
SOLVE    1
PIK      1
TRC      1
SIL V    1
PRU      1
GEAY     1
M.GRE    1
WHTQ     1
STRLT    1
BLDG     1
RD-CH    1
ELK      1
WICK     1
MARIO    1
CH       1
Name: Vehicle Color, dtype: int64

In [139]:
# how can I fix this kind of problem?

df['Vehicle Color'].value_counts().head(20)

WH       2344858
GY       2307704
BK       2066374
WHITE    1061234
BL        775124
RD        483298
BLACK     465110
GREY      306787
BROWN     292348
SILVE     191477
GR        182929
BLUE      178298
RED       161693
TN        120576
BR        102204
YW         98700
BLK        91539
OTHER      60245
GREEN      58765
GL         54851
Name: Vehicle Color, dtype: int64

In [140]:
# get the vehicle color for all cars that have a color of 'WH'
df.loc[df['Vehicle Color'] == 'WH', 'Vehicle Color']

6           WH
16          WH
18          WH
19          WH
21          WH
            ..
12495684    WH
12495693    WH
12495698    WH
12495712    WH
12495717    WH
Name: Vehicle Color, Length: 2344858, dtype: object

In [141]:
# get the vehicle color for all cars that have a color of 'WH', and assign them to be white
df.loc[df['Vehicle Color'] == 'WH', 'Vehicle Color'] = 'WHITE'

In [142]:
df['Vehicle Color'].value_counts().head(20)

WHITE    3406092
GY       2307704
BK       2066374
BL        775124
RD        483298
BLACK     465110
GREY      306787
BROWN     292348
SILVE     191477
GR        182929
BLUE      178298
RED       161693
TN        120576
BR        102204
YW         98700
BLK        91539
OTHER      60245
GREEN      58765
GL         54851
GRY        46527
Name: Vehicle Color, dtype: int64

In [143]:
# replace two-letter colors with longer color names

#     row selector w/boolean index   , column selector  = assignment
df.loc[df['Vehicle Color'] == 'GY', 'Vehicle Color'] = 'GREY'
df.loc[df['Vehicle Color'] == 'BK', 'Vehicle Color'] = 'BLACK'
df.loc[df['Vehicle Color'] == 'BL', 'Vehicle Color'] = 'BLUE'
df.loc[df['Vehicle Color'] == 'RD', 'Vehicle Color'] = 'RED'
df.loc[df['Vehicle Color'] == 'GR', 'Vehicle Color'] = 'GREEN'
df.loc[df['Vehicle Color'] == 'TN', 'Vehicle Color'] = 'TAN'
df.loc[df['Vehicle Color'] == 'BR', 'Vehicle Color'] = 'BROWN'
df.loc[df['Vehicle Color'] == 'YW', 'Vehicle Color'] = 'YELLO'

In [144]:
df['Vehicle Color'].value_counts().head(20)

WHITE    3406092
GREY     2614491
BLACK    2531484
BLUE      953422
RED       644991
BROWN     394552
GREEN     241694
SILVE     191477
TAN       141667
YELLO     131492
BLK        91539
OTHER      60245
GL         54851
GRY        46527
MR         42812
GRAY       40854
WHT        35433
WHI        29760
OR         28100
BK.        27830
Name: Vehicle Color, dtype: int64

In [147]:
df.count()   # how many non-nan values are there in each column?

Plate ID              12495532
Registration State    12495734
Vehicle Make          12433314
Violation Time        12495456
Street Name           12494317
Vehicle Color         12103752
dtype: int64

In [148]:
df.shape

(12495734, 6)

In [149]:
# this will tell me how many non-nan values I have per columns
df.shape[0] - df.count()

Plate ID                 202
Registration State         0
Vehicle Make           62420
Violation Time           278
Street Name             1417
Vehicle Color         391982
dtype: int64

In [150]:
# If New York makes $100 per parking ticket, then if we remove all of the nan values,
# how much money would the city lose?

391982 * 100  # $39m!

39198200

In [151]:
# keep rows in which we have at least 3 columns of data
df.dropna(thresh=3).shape

(12495730, 6)

In [152]:
df.shape[0] - 12495730

4

In [153]:
df.dropna(thresh=4).shape

(12495524, 6)

In [154]:
df.shape[0] - 12495524

210

In [155]:
df = DataFrame([['Reuven', 'Lerner', 51],
                ['Atara', 'Lerner-Friedman', 21],
                ['Shikma', 'Lerner-Friedman', 19],
                ['Amotz', 'Lerner-Friedman', 16],
                ['John', 'Smith', 35],
                ['David', 'Cohen', 60],
                ['Sarah', 'Friedman', 59]                
               ],
              columns='firstname lastname age'.split())     # ['firstname', 'lastname', 'age']

In [156]:
df

,firstname,lastname,age
0,Reuven,Lerner,51
1,Atara,Lerner-Friedman,21
2,Shikma,Lerner-Friedman,19
3,Amotz,Lerner-Friedman,16
4,John,Smith,35
5,David,Cohen,60
6,Sarah,Friedman,59


In [157]:
# I want to add a new column, age_category.  This category will describe people 
# as young, midage, or old.  (So that I'll have some people in each category, I'll
# define young as under 20, 20-50 is midage, and old is above 50.)

# how can I do that?

# option 1: define a new column in which everyone is midage.
# then assign 'young' to anyone with an age < 20 and 'old' to anyone with an age > 50

df['age_category'] = 'midage'    # broadcasting this assignment to the new column

In [158]:
df

,firstname,lastname,age,age_category
0,Reuven,Lerner,51,midage
1,Atara,Lerner-Friedman,21,midage
2,Shikma,Lerner-Friedman,19,midage
3,Amotz,Lerner-Friedman,16,midage
4,John,Smith,35,midage
5,David,Cohen,60,midage
6,Sarah,Friedman,59,midage


In [160]:
df.loc[df['age'] < 20, 'age_category'] = 'young'
df.loc[df['age'] > 50, 'age_category'] = 'old'

In [161]:
df

,firstname,lastname,age,age_category
0,Reuven,Lerner,51,old
1,Atara,Lerner-Friedman,21,midage
2,Shikma,Lerner-Friedman,19,young
3,Amotz,Lerner-Friedman,16,young
4,John,Smith,35,midage
5,David,Cohen,60,old
6,Sarah,Friedman,59,old


In [162]:
df['age_category'].value_counts()

old       3
midage    2
young     2
Name: age_category, dtype: int64

In [163]:
# this turns out to be very common -- take continuous data (i.e., numbers) and turn it into
# categorical data.  

df = DataFrame([['Reuven', 'Lerner', 51],
                ['Atara', 'Lerner-Friedman', 21],
                ['Shikma', 'Lerner-Friedman', 19],
                ['Amotz', 'Lerner-Friedman', 16],
                ['John', 'Smith', 35],
                ['David', 'Cohen', 60],
                ['Sarah', 'Friedman', 59]                
               ],
              columns='firstname lastname age'.split())     # ['firstname', 'lastname', 'age']



In [166]:
help(pd.cut)

Help on function cut in module pandas.core.reshape.tile:

cut(x, bins, right: 'bool' = True, labels=None, retbins: 'bool' = False, precision: 'int' = 3, include_lowest: 'bool' = False, duplicates: 'str' = 'raise', ordered: 'bool' = True)
    Bin values into discrete intervals.
    
    Use `cut` when you need to segment and sort data values into bins. This
    function is also useful for going from a continuous variable to a
    categorical variable. For example, `cut` could convert ages to groups of
    age ranges. Supports binning into an equal number of bins, or a
    pre-specified array of bins.
    
    Parameters
    ----------
    x : array-like
        The input array to be binned. Must be 1-dimensional.
    bins : int, sequence of scalars, or IntervalIndex
        The criteria to bin by.
    
        * int : Defines the number of equal-width bins in the range of `x`. The
          range of `x` is extended by .1% on each side to include the minimum
          and maximum values 

In [167]:
pd.cut(df['age'], 
      bins=[0, 20, 50, 100],               # boundaries before/between/after categories
      labels=['young', 'midage', 'old'])   # number of labels = len(bins) - 1

0       old
1    midage
2     young
3     young
4    midage
5       old
6       old
Name: age, dtype: category
Categories (3, object): ['young' < 'midage' < 'old']

In [168]:
df['age_category'] = pd.cut(df['age'], 
      bins=[0, 20, 50, 100],               # boundaries before/between/after categories
      labels=['young', 'midage', 'old'])   # number of labels = len(bins) - 1

In [169]:
df

,firstname,lastname,age,age_category
0,Reuven,Lerner,51,old
1,Atara,Lerner-Friedman,21,midage
2,Shikma,Lerner-Friedman,19,young
3,Amotz,Lerner-Friedman,16,young
4,John,Smith,35,midage
5,David,Cohen,60,old
6,Sarah,Friedman,59,old


# Exercise: Short, medium, and long taxi rides

1. Load the taxi data from January 2019 in the new zipfile you downloaded today.  Once again, we're only interested in `trip_distance`, `passenger_count`, and `total_amount`.
2. Show the number of rides in each of these three categories:
    - short (<= 2 miles)
    - medium (> 2 miles, but <= 10 miles)
    - long (> 10 miles)
3. For each category, show the average total amount and passenger count.
    - Based on this, would you say that longer trips cost more?
    - Based on this, would you say that longer trips have more passengers?

In [170]:
!ls ../data/*2019*

../data/nyc_taxi_2019-01.csv  ../data/nyc_taxi_2019-07.csv
